In [6]:
from snowflake.snowpark import Session
from snowflake.ml.utils import connection_params

session = Session.builder.configs(connection_params.SnowflakeLoginOptions()).create()

#Clean up old objects
[session.sql(f"DROP DATABASE {db['name']}").collect()  for db in session.sql("SHOW DATABASES").collect() if db['name'].startswith('SNOWFLAKE_MLPLATFORM_')]

[[Row(status='SNOWFLAKE_MLPLATFORM_TEST_E0679AB3 successfully dropped.')]]

In [7]:
#Create unique ephemeral namespace
from datetime import datetime
import uuid

SNOWFLAKE_DATABASE = f"SNOWFLAKE_MLPLATFORM_TEST_{uuid.uuid4().hex[:8].upper()}"
SNOWFLAKE_SCHEMA = f"SNOWFLAKE_MLPLATFORM_TEST_{datetime.now().strftime('%Y%m%d%H%M%S')}_{uuid.uuid4().hex[:8].upper()}"
SNOWFLAKE_DATABASE_PROD = "SNOWFLAKE_ML_PLATFORM"
SNOWFLAKE_SCHEMA_PROD = "PROD"

#Create ephemeral database to land prod clone
session.sql(f"CREATE DATABASE {SNOWFLAKE_DATABASE}").collect(); session.sql("DROP SCHEMA PUBLIC").collect()

#Clone prod environment
session.sql(f"CREATE SCHEMA {SNOWFLAKE_DATABASE}.{SNOWFLAKE_SCHEMA} CLONE {SNOWFLAKE_DATABASE_PROD}.{SNOWFLAKE_SCHEMA_PROD}").collect()

#Bandaid to copy models
from migrations.models.clone_models import clone_models
clone_models(session, SNOWFLAKE_DATABASE_PROD, SNOWFLAKE_SCHEMA_PROD)

In [4]:
from migrations.features.apply_features import apply_feature_changes
from migrations.datasets.apply_datasets import apply_dataset_changes
from migrations.models.apply_models import apply_model_changes

apply_feature_changes(session)
apply_dataset_changes(session)
apply_model_changes(session)